<a href="https://colab.research.google.com/github/janShi1105/science/blob/main/ML17_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np

if tf.config.list_physical_devices('GPU'):
  device_name = tf.test.gpu_device_name()
else:
  device_name = '/CPU:0'
print(device_name)

/device:GPU:0


In [4]:
def make_dcgan_generator(z_size=20, output_size= (28,28,1), n_filters=128, n_blocks=2):
  size_factor = 2**n_blocks
  hidden_size = (output_size[0]//size_factor, output_size[1]//size_factor)

  model = tf.keras.Sequential([
      tf.keras.layers.Input(shape=(z_size, )),
      tf.keras.layers.Dense(units=n_filters * np.prod(hidden_size), use_bias = False),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(),
      tf.keras.layers.Reshape((hidden_size[0], hidden_size[1], n_filters)),
      tf.keras.layers.Conv2DTranspose(filters=n_filters, kernel_size=(5,5), strides=(1,1), padding='same', use_bias=False),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU()
  ])

  nf = n_filters
  for i in range(n_blocks):
    nf = nf //2
    model.add(tf.keras.layers.Conv2DTranspose(filters=nf, kernel_size=(5,5), strides=(2,2), padding='same', use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

  model.add(tf.keras.layers.Conv2DTranspose(filters=output_size[2], kernel_size=(5,5), strides=(1,1), padding='same', use_bias=False, activation='tanh'))

  return model


In [3]:
def make_dcgan_discriminator(input_size=(28,28,1), n_filters=64, n_blocks=2):
  model = tf.keras.Sequential([
      tf.keras.layers.Input(shape=input_size), 
      tf.keras.layers.Conv2D(filters=n_filters, kernel_size=5, strides=(1,1), padding='same'),
      tf.keras.layers.BatchNormalization(), 
      tf.keras.layers.LeakyReLU()
  ])

  nf = n_filters 
  for i in range(n_blocks):
    nf = nf * 2
    model.add(tf.keras.layers.Conv2D(filters=nf, kernel_size=(5,5), strides=(2,2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))

  model.add(tf.keras.layers.Conv2D(filters=1, kernel_size=(7,7), padding='valid'))
  model.add(tf.keras.layers.Reshape((1,)))

  return model

In [5]:
import tensorflow_datasets as tfds

mnist_bldr = tfds.builder('mnist')
mnist_bldr.download_and_prepare()
mnist = mnist_bldr.as_dataset(shuffle_files=False)

def preprocess(ex, mode='uniform'):
  image = ex['image']
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = image * 2 - 1
  if mode == 'uniform':
    input_z = tf.random.uniform(shape=(z_size, ), minval=-1, maxval=1)
  elif mode == 'normal':
    input_z = tf.random.normal(shape=(z_size,))

  return input_z, image

Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to ~/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [6]:
gen_model = make_dcgan_generator()
gen_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6272)              125440    
                                                                 
 batch_normalization (BatchN  (None, 6272)             25088     
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 6272)              0         
                                                                 
 reshape (Reshape)           (None, 7, 7, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 7, 7, 128)        409600    
 nspose)                                                         
                                                                 
 batch_normalization_1 (Batc  (None, 7, 7, 128)        5

In [8]:
disc_model = make_dcgan_discriminator()
disc_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 28, 28, 64)        1664      
                                                                 
 batch_normalization_7 (Batc  (None, 28, 28, 64)       256       
 hNormalization)                                                 
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 28, 28, 64)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 14, 14, 128)       204928    
                                                                 
 batch_normalization_8 (Batc  (None, 14, 14, 128)      512       
 hNormalization)                                                 
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 14, 14, 128)      

In [9]:
num_epochs  = 100
batch_size = 128
image_size = (28,28)
z_size = 20
mode_z = 'uniform'
lambda_gp = 10

tf.random.set_seed(1)
np.random.seed(1)

mnist_trainset = mnist['train']
mnist_trainset = mnist_trainset.map(preprocess)
mnist_trainset = mnist_trainset.shuffle(10000)
mnist_trainset = mnist_trainset.batch(batch_size, drop_remainder=True)

with tf.device(device_name):
  gen_model = make_dcgan_generator()
  gen_model.build(input_shape=(None, z_size))
  disc_model = make_dcgan_discriminator()
  disc_model.build(input_shape=(None, np.prod(image_size)))

In [10]:
import time

g_optimizer = tf.keras.optimizers.Adam(0.0002)
d_optimizer = tf.keras.optimizers.Adam(0.0002)

if mode_z == 'uniform':
  fixed_z = tf.random.uniform(shape=(batch_size, z_size), minval=-1, maxval =1)
elif mode_z == 'normal':
  fixed_z = tf.random.normal(shape=(batch_size, z_size))

def create_sample(g_model, input_z):
  g_output = g_model(input_z, training=False)
  images = tf.reshape(g_output, (batch_size, *image_size))
  return (images + 1)/2


In [ ]:
all_losses = []
epoch_samples = []
start_time = time.time()

for epoch in range(1, num_epochs + 1):
  epoch_losses  = []
  for i, (input_z , input_real) in enumerate(mnist_trainset):
    with tf.GradientTape() as d_type, tf.GradientTape() as g_tape:
      g_output = gen_model(input_z, training=True)
      d_critics_real = disc_model(input_real, training = True)
      d_critics_fake = disc_model(g_output, training =True)

      g_loss = -tf.math.reduce_mean(d_critics_fake)

      d_loss_real = -tf.math.reduce_mean(d_critics_real)
      d_loss_fake = tf.math.reduce_mean(d_critics_fake
      d_loss = d_loss_real + d_loss_fake

      with tf.GradientTape() as gp_tape:
        alpha = tf.random.uniform(shape=[d_critics_real.shape[0], 1,1,1], minval=0, maxval=1)
        interpolated = (alpha * input_real + (1-alpha) * g_output)
        gp_tape.watch(interpolated)
        d_critics_intp = disc_model(interpolated)

      grads_intp = gp_tape.gradient(d_critics_intp, [interpolated,])[0]
      grads_intp_l2 = tf.sqrt(tf.reduce_mean(tf.square(grads_intp), axis=[1,2,3]))
      grad_penalty = tf.reduce_mean(tf.square(grads_intp_l2 -1))
      d_loss = d_loss + lambda_gp * grad_penalty
    d_grads = d_tape.gradient(d_loss, dics_model.trainable_variables)
    d_optimizer.apply_gradients(
        grads_and_vars=zip(d_grads, dics_model.trainable_variables)
    )
    g_grads = g_tape.gradient(g_loss, gen_model.trainable_variables)
    g_optimizer.apply_gradients(grads_and_vars=zip(g_grads, gen_model.trainable_variables))
    epoch_losses.append((g_loss.numpy(), d_loss.numpy(), d_loss_real.numpy(), d_loss_fake.numpy()))

  all_losses.append(epoch_losses)
  print('Epoch {:.3d} | ET {:.2f} min | Avg Losses >> G/D {:6.2f}/{:6.2f} [D-Real: {:6.2f}  D-Fake: {:6.2f}]'.format(epoch, (time.time() -start_time)/60, *list(np.mean(all_losses[-1], axis=0))))
  epoch_samples.append(create_samples(gen_model, fixed_z).numpy())